# Capturing and Exploring Data from the RTL SDR

NOTES:
+ You need an RTL SDR dongle to capture data. They can be purchased for about \$20 on Amazon.
+ Data capture requires the **pyrtlsdr** library. Install with pip.
+ Saving the data in SigMF format requires the **sigmf** library, which can be found on github [here](https://github.com/gnuradio/SigMF)
+ `read_samples()` cannot capture an arbitrary number of samples. 4 seconds is about the maximum I am able to read without erroring out
+ If you get an error when trying to read samples, chances are you will need to restart the IPython kernel

References:
+ https://witestlab.poly.edu/blog/capture-and-decode-fm-radio/
+ https://tomroelandts.com/articles/how-to-create-a-simple-low-pass-filter
+ https://en.wikipedia.org/wiki/Heterodyne
+ https://en.wikipedia.org/wiki/Downsampling_(signal_processing)

In [ ]:
%matplotlib inline

In [ ]:
from rtlsdr import RtlSdr
import cmath
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
from IPython.display import Audio
import datetime as dt

In [ ]:
help(RtlSdr)

## Using pyrtlsdr to interact with the device

In [ ]:
#Get a list of detected devices (how many dongles do you have plugged in?)
serial_numbers = RtlSdr.get_device_serial_addresses()
print("List of Serial Numbers: {}".format(serial_numbers)) # Default serial number for all Rafael Micro R820T/2 chips is "1"

# You can interact with the device by serial number, or you can also use a "device index"
device_index = RtlSdr.get_device_index_by_serial('00000001') # Find the device index for a given serial number
print("Device Index for SN {}: {}".format('00000001', device_index))

In [ ]:
# Instantiate an RtlSdrAio object
sdr = RtlSdr(serial_number='00000001')

In [ ]:
# Check some properties
print(sdr.get_tuner_type())
print(sdr.get_gains())

# Configure the dongle
sdr.sample_rate = 2.048e6 # samples/second
sdr.center_freq = 100e6 # Hz (The Rafael Micro R820T/2 chipset supports 24 - 1766 MHz)
#sdr.freq_correction = 1 # ppm
sdr.gain = "auto" # dB

In [ ]:
# Confirm settings
print("Sample rate is: {} samples/sec".format(sdr.get_sample_rate()))
print("Center frequency is set to: {} Hz".format(sdr.get_center_freq()))
print("Frequency offset correction set to: {} parts per million".format(sdr.get_freq_correction()))
print("Gan: {} dB (0.0 = automatic gain control".format(sdr.get_gain()))
print("Bandwidth is: {}".format(sdr.get_bandwidth()))

In [ ]:
help(sdr.read_samples)

In [ ]:
# Capture data for one 2048-point FFT and PSD plot
samples = sdr.read_samples(2048)

Plot the [**Power Spectral Density (PSD)**](https://en.wikipedia.org/wiki/Spectral_density) using pyplot's builtin function. This will give us an idea of what signals are out there.
+ See [this reference page](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.psd.html)

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))

Pxx, freqs = ax.psd(x=samples,
                    NFFT=2048,
                    Fs=sdr.sample_rate,
                    Fc=sdr.center_freq,
                    detrend=None,
                    window=None,
                    noverlap=None,
                    pad_to=None,
                    sides='twosided',
                    scale_by_freq=None,
                    return_line=None)

In [ ]:
# Capture 4 seconds of data and plot using specgram
timestamp = dt.datetime.now() # we will use this later
samples = sdr.read_samples(4 * sdr.sample_rate)
Fs = sdr.get_sample_rate()
Fc = sdr.get_center_freq()

fig, ax = plt.subplots(figsize=(16,12))
_ = ax.specgram(x=samples,
             NFFT=2048,
             Fs=Fs,
             Fc=Fc,
             detrend=None,
             window=None, 
             noverlap=None,
             cmap=None,
             xextent=None,
             pad_to=None,
             sides='twosided',
             scale_by_freq=None,
             mode=None,
             scale=None)

I see a two traces of energy within this ~2 MHz bandwidth.  One at 100.5 MHz and one at 99.7 MHz, corresponding to FM radio stations in my area. Your results will be different based on what radio stations are present in your environment.

I am interested in the trace at 99.7 MHz, since it looks stronger. First, I want to "shift" the 99.7 MHz signal to the center of my bandwidth (i.e., "baseband"). It will help if we stop applying the frequency correction labels and consider the actual frequency components of our sampled data.

In [ ]:
fig, ax = plt.subplots(figsize=(16,5))

Pxx, freqs = ax.psd(x=samples,
                    NFFT=2048,
                    Fs=sdr.sample_rate,
                    Fc=0, # notice that I've adjusted this to zero, which actually changes nothing except the labels
                    detrend=None,
                    window=None,
                    noverlap=None,
                    pad_to=None,
                    sides='twosided',
                    scale_by_freq=None,
                    return_line=None)

Now I am viewing the PSD plot without the adjusted frequency labels. I can see that my trace of interest is centered at -3e5 Hz (-300 KHz), or 300 kHz to the left of baseband. I want to adjust this up to 0 Hz (this is called **"basebanding"** the signal, or bringing it "down to baseband"). The way we do this is a process called **"heterodyning"**, which is a fancy name for something that is actually fairly simple, especially when working with complex data.

[**Heterodyning**](https://en.wikipedia.org/wiki/Heterodyne) is simply multiplying the signal with a pure sinusoid. This has the effect of "shifting" all of the frequency content. You can see how this works mathematically:

$$
e^{j2{\pi}f_1t}{\cdot}e^{j2{\pi}f_2t} = e^{j2{\pi}(f_1+f_2)t}
$$

So, if $f_1 = -300$ kHz is the frequency of my signal, and I want to shift it to $f_1 + f_2 = 0.0$, then I need $f_2$ to be $0.0 - (-300) = +300$ kHz.

In [ ]:
# "upconvert" the -300 kHz signal to center by mixing with a 3e5 Hz sinusoid
f2 = -5.0e5 # Hz
# create a pure sinusoid at freq = f2
t = np.arange(0,len(samples))/sdr.sample_rate # time steps (sample_period = 1/sample_rate)
mixing_signal = np.exp(1j*2*np.pi*f2*t) # complex sinusoid at 300 kHz

shifted = samples * mixing_signal

In [ ]:
# Plot PSD to make sure it worked
fig, ax = plt.subplots(figsize=(16,5))

Pxx, freqs = ax.psd(x=shifted,
                    NFFT=2048,
                    Fs=sdr.sample_rate,
                    Fc=0,
                    detrend=None,
                    window=None,
                    noverlap=None,
                    pad_to=None,
                    sides='twosided',
                    scale_by_freq=None,
                    return_line=None)

Now I want to filter out all the other frequencies I don't care about. I can do this using a **Low Pass Filter**, a very handy signal processing technique.

Low-pass filter reference:
+ https://tomroelandts.com/articles/how-to-create-a-simple-low-pass-filter

For a cutoff frequency of 100 kHz (which will result in a 200 MHz two-sided bandwidth):

$$
\frac{200000}{F_s} = \frac{f_{cutoff-fractional}}{0.5}
$$

In [ ]:
# Create a low-pass filter
cutoff = 150.0e3 # cutoff frequency
frac = cutoff/sdr.sample_rate
N = int(np.ceil(4/.02)) # number of filter taps (coefficients)
if N % 2: N += 1
w = np.blackman(N)
sinc = np.sinc(2 * frac * (np.arange(N) - (N-1)/2))
h = sinc * w

# Apply the filter
filtered = np.convolve(shifted, h)

# Plot the spectrogram
fig, ax = plt.subplots(figsize=(16,8))
_ = ax.specgram(x=filtered,
             NFFT=2048,
             Fs=sdr.sample_rate,
             Fc=sdr.center_freq,
             detrend=None,
             window=None, 
             noverlap=None,
             cmap=None,
             xextent=None,
             pad_to=None,
             sides='twosided',
             scale_by_freq=None,
             mode=None,
             scale=None)

In [ ]:
# Plot the PSD
fig, ax = plt.subplots(figsize=(16,5))
Pxx, freqs = ax.psd(x=filtered[:200 * 2048],
                    NFFT=2048,
                    Fs=sdr.sample_rate,
                    Fc=sdr.center_freq,
                    detrend=None,
                    window=None,
                    noverlap=None,
                    pad_to=None,
                    sides='twosided',
                    scale_by_freq=None,
                    return_line=None)

It looks like the low-pass filter worked, so now I am left with just the signal I am interested in, centered at baseband. However, the sample rate is still very high, and if I try to convert it into an audio signal it won't work. I need to reduce the sample rate using a process called [**downsampling**](https://en.wikipedia.org/wiki/Downsampling_(signal_processing)), also called **decimation**.

**Downsampling** is much easier if you downsample by an integer factor. There are a number of different methods for downsampling, each having its pros/cons. I will use the simplest downsampling method by just taking every $n^{th}$ sample and throwing the rest out.

In [ ]:
# Downsample ("decimate") to a sample rate within range of sound card (44.1 KHz is typical digital audio sample rate)
r = 10 # decimation rate
if len(filtered)%r > 0:
    filtered = filtered[:-(len(filtered)%r)] # truncate to a multiple of the decimation rate
print("Decimating to {} samples/sec".format(sdr.sample_rate/r))
#downsampled = np.mean(filtered.reshape(-1,r),1) # downsample by taking average of every 'r' samples
downsampled = filtered[::r] # downsample by throwing out all except every 'r' samples

In [ ]:
# Take a look at our new decimated baseband signal in a spectrogram
fig, ax = plt.subplots(figsize=(16,5))
_ = ax.specgram(x=downsampled,
             NFFT=256,
             Fs=sdr.sample_rate/r,
             Fc=0,
             detrend=None,
             window=None, 
             noverlap=None,
             cmap=None,
             xextent=None,
             pad_to=None,
             sides='twosided',
             scale_by_freq=None,
             mode=None,
             scale=None)

In [ ]:
# Plot the PSD
fig, ax = plt.subplots(figsize=(16,5))
Pxx, freqs = ax.psd(x=downsampled,
                    NFFT=2048,
                    Fs=sdr.sample_rate/r,
                    Fc=0,
                    detrend=None,
                    window=None,
                    noverlap=None,
                    pad_to=None,
                    sides='twosided',
                    scale_by_freq=None,
                    return_line=None)

Complex baseband samples are sometimes viewed in a [**constellation diagram**](https://en.wikipedia.org/wiki/Constellation_diagram#:~:text=A%20constellation%20diagram%20is%20a,plane%20at%20symbol%20sampling%20instants.), where the *in-phase (I)* dimension (i.e., real part) of each sample is plotted against the *quadrature (Q)* dimension (i.e., imaginary part).

This results in a projection (flattening) of the time dimension onto the complex plane, and can be thought of as "looking down the barrel" of the signal.

In [ ]:
# Plot the constellation for 1/100th of a second
plt.scatter(np.real(downsampled[:2048]), np.imag(downsampled[:2048]), color="red", alpha=0.05)  

To **demodulate an FM signal**, we need to compute the instantaneous frequency of the carrier at each sample. Remember that the frequency of a signal is simply how fast the phase term changes (i.e., how many times the complex representation rotates around the origin every second). So frequency is actually the first derivative of phase:

$$
f = \frac{d{\Theta}}{dt}
$$

So, to compute an estimate of instantaneous frequency, we simply need to compute the phase at each sample, and subtract the phase at the previous sample:

$$
f = \frac{d{\Theta}}{dt} = \frac{{\Theta}_2 - {\Theta}_1}{{\Delta}t}
$$

...And, since we don't really care about scaling (scaling only raises or lowers the gain (think volume)), we can actually throw out the ${\Delta}t$ term:

$$
f {\propto} [{\Theta}_2 - {\Theta}_1]
$$

Now, it seems like it should be easy to just compute the phase angle of every sample, then take the difference between subsequent samples:

In [ ]:
angles = np.angle(downsampled)
diffs = np.diff(angles)

Audio(diffs, rate=sdr.sample_rate/r, autoplay=True)

...but, it just sounds like static.  We'll find there's an issue here, which I will point out in a second. Instead, we will use a technique called a **polar discriminator** to compute the difference between subsequent phase angles.

This technique consists of rotating each sample by the opposite phase angle for the previous sample. In the complex plane, rotation is accomplished by multiplication with a complex number with a phase angle equal to the desired rotation, and magnitude 1:

In [ ]:
# Rotate (1 + 1j) by pi/6 radians

s1 = 1 + 1j
xs = [0,np.real(s1)]
ys = [0,np.imag(s1)]
plt.plot(xs, ys, label="s1")

phase = np.pi/6
radius = 1
s2 = cmath.rect(radius, phase)
xs = [0, np.real(s2)]
ys = [0, np.imag(s2)]
plt.plot(xs,ys,label="s2 (rotator)")

s3 = s1 * s2
xs = [0,np.real(s3)]
ys = [0,np.imag(s3)]
plt.plot(xs, ys, label="product")
print("Angle of s1: {},\nAngle of s2: {},\nAngle of product: {}".format(np.angle(s1), np.angle(s2), np.angle(s3)))

s4 = s1 * np.conj(s2) # multiplying by complex conjugate subtracts the phase angle
xs = [0,np.real(s4)]
ys = [0,np.imag(s4)]
plt.plot(xs, ys, label="product with conjugate")

plt.xlim((-2,2))
plt.ylim((-2,2))
plt.legend()
plt.gca().set_aspect('equal')
print(np.absolute(s4))

So, we can see that by multiplying each sample by the complex conjugate of the previous sample, we will get a series of complex numbers whose phase angles represent the differences between consecutive phase angles

In [ ]:
# FM Demodulate

# counter-rotate each sample by the previous sample's phase angle
rotated = downsampled[1:] * np.conj(downsampled[:len(downsampled)-1])

# compute phase angle (this will equal the difference in phase between subsequent samples)
deltas = np.angle(rotated)

In [ ]:
Audio(deltas, rate=sdr.sample_rate/r, autoplay=True)

So, why all this additional mathematical jiu-jitsu with multiplying by the complex conjugate?  Why can't we just take a simple diff between angle representations of the samples?

Let's take a look at what that provides:

In [ ]:
plt.plot(diffs[:50], label='straight differences')
plt.plot(deltas[:50], label='jiu jitsu')
plt.legend()

So, it looks like the phase differences are equal most of the time, but every so often they are way off. What's going on here?

In [ ]:
i = 0
while diffs[i] - deltas[i] < 1e-10:
    i += 1
print(diffs[i])
print(deltas[i])

diffs[i] - deltas[i]

What we notice here is that the phase angle is off by $2\pi$, which means the phase angles are actually equal! There is no difference, except that when you compute the phase angle first, then take the difference, it is possible to get a resulting phase angle anywhere within \($-2\pi$,$2\pi$\).  When you use the polar discriminator, the final angle is computed after the rotation, and will only give angles within ($-\pi$, $\pi$).

We can fix this with a DSP technique called ["phase wrapping"](https://en.wikipedia.org/wiki/Instantaneous_phase_and_frequency).

In [ ]:
adjusted_diffs = np.zeros(len(diffs))
for i, angle in np.ndenumerate(diffs):
    if angle > np.pi:
        adjusted_diffs[i] = angle - 2*np.pi
    elif angle < -np.pi:
        adjusted_diffs[i] = angle + 2*np.pi
    else:
        adjusted_diffs[i] = angle

In [ ]:
plt.plot(adjusted_diffs[:50], label="adjusted diffs")
plt.plot(deltas[:50], label="deltas")
plt.legend()

Now we see that our results are the same as the polar discriminator. Let's listend and see:

In [ ]:
Audio(adjusted_diffs, rate=sdr.sample_rate/r, autoplay=True)

What if we want to retrieve this capture later?  Let's save the raw data and use the [SigMF metadata format](https://github.com/gnuradio/SigMF) to store the details--things that someone else might need to know in order to use the data:
+ frequency
+ sample rate
+ data type and format

Along with other data that might be useful for people to know:
+ date and time recorded
+ equipment used, etc...

We will need to install the SigMF python modules:

```console
$ git clone https://github.com/gnuradio/SigMF.git
$ cd SigMF
$ python -m pip install .
```

In [ ]:
import sigmf
from sigmf import SigMFFile

In [ ]:
samples.dtype

In [ ]:
sdr.get_sample_rate()

In [ ]:
# specify filename to save the data:
filename = 'FM-radio-sample-data.sigmf-data' # per the SigMF spec, the raw data has the '.sigmf-data' extension

# SigMF core module only supports single-precision floats, need to convert to 32-bit I and 32-bit Q:
temp = samples.astype(np.complex64)
# use numpy's built-in function to write an array to a file:
temp.tofile(filename)

In [ ]:
# create the metadata:
meta = SigMFFile(data_file=filename,
                global_info={
                    SigMFFile.DATATYPE_KEY: 'cf32_le', # complex floating point, little endian
                    SigMFFile.SAMPLE_RATE_KEY: Fs,
                    SigMFFile.AUTHOR_KEY: 'britishtar@yahoo.com',
                    SigMFFile.DESCRIPTION_KEY: 'FM Radio Recording for DSP tutorial',
                    SigMFFile.VERSION_KEY: sigmf.__version__,
                })

# this is a "capture key" for time index 0
meta.add_capture(0, metadata={
    SigMFFile.FREQUENCY_KEY: Fc,
    SigMFFile.DATETIME_KEY: timestamp.isoformat()+'Z',
})

# this is an annotation for the FM radio trace at 99.7 MHz
meta.add_annotation(0, len(samples), metadata={
    SigMFFile.FLO_KEY: (99.7e6 - 100e3),
    SigMFFile.FHI_KEY: (99.7e6 + 100e3),
    SigMFFile.COMMENT_KEY: "K-Ci and JoJo - 'All My Life'",
})

# this is an annotation for the FM radio trace at 100.5 MHz
meta.add_annotation(0, len(samples), metadata={
    SigMFFile.FLO_KEY: (100.5e6 - 100e3),
    SigMFFile.FHI_KEY: (100.5e6 + 100e3),
    SigMFFile.COMMENT_KEY: "A random advertisement",
})

# write the metadata to disk
meta.tofile('FM-radio-sample-data.sigmf-meta')

# validate the metadata tag
assert meta.validate(), print(meta.validate())

This metadata fails validation because it doesn't think the annotations are ordered properly.  However, I intended them to annotate the entire length of the clip, so there really isn't an order.  The [SigMF spec](https://github.com/gnuradio/SigMF/blob/master/sigmf-spec.md#annotation-segment-objects) even states:

"If two annotations have the same `sample_start`, there is no defined ordering between them."

So I think this is fine.

In [ ]:
np.complex128

In [ ]:
samples = np.fromfile('FM-radio-sample-data.sigmf-data', dtype=np.complex64)

In [ ]:
fig, ax = plt.subplots(figsize=(16,12))
_ = ax.specgram(x=samples,
             NFFT=2048,
#              Fs=Fs,
#              Fc=Fc,
             detrend=None,
             window=None, 
             noverlap=None,
             cmap=None,
             xextent=None,
             pad_to=None,
             sides='twosided',
             scale_by_freq=None,
             mode=None,
             scale=None)

In [ ]:
# "upconvert" the -300 kHz signal to center by mixing with a 3e5 Hz sinusoid
f2 = 3.0e5 # Hz
# create a pure sinusoid at freq = f2
t = np.arange(0,len(samples))/sdr.sample_rate # time steps (sample_period = 1/sample_rate)
mixing_signal = np.exp(1j*2*np.pi*f2*t) # complex sinusoid at 300 kHz

shifted = samples * mixing_signal

# Create a low-pass filter
cutoff = 150.0e3 # cutoff frequency
frac = cutoff/sdr.sample_rate
N = int(np.ceil(4/.02)) # number of filter taps (coefficients)
if N % 2: N += 1
w = np.blackman(N)
sinc = np.sinc(2 * frac * (np.arange(N) - (N-1)/2))
h = sinc * w

# Apply the filter
filtered = np.convolve(shifted, h)

# Downsample ("decimate") to a sample rate within range of sound card (44.1 KHz is typical digital audio sample rate)
r = 10 # decimation rate
if len(filtered)%r > 0:
    filtered = filtered[:-(len(filtered)%r)] # truncate to a multiple of the decimation rate
print("Decimating to {} samples/sec".format(sdr.sample_rate/r))
#downsampled = np.mean(filtered.reshape(-1,r),1) # downsample by taking average of every 'r' samples
downsampled = filtered[::r] # downsample by throwing out all except every 'r' samples

# FM Demodulate

# counter-rotate each sample by the previous sample's phase angle
rotated = downsampled[1:] * np.conj(downsampled[:len(downsampled)-1])

# compute phase angle (this will equal the difference in phase between subsequent samples)
deltas = np.angle(rotated)

In [ ]:
Audio(deltas, rate=sdr.sample_rate/r, autoplay=True)

In [ ]:
# TODO: decode RBDS data

# TODO: isolate/demod left and right stereo channels